<a href="https://colab.research.google.com/github/rohithreddyvennam/AssignmentsFiles/blob/master/EE18MECH11026_EE5611_A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Statements

In [0]:
# All import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Read From CSV File


In [0]:
# Read all CSV Files
train_df = pd.read_csv('https://raw.githubusercontent.com/rohithreddyvennam/AssignmentsFiles/master/train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/rohithreddyvennam/AssignmentsFiles/master/test_sample.csv')
submission_df = pd.read_csv('https://raw.githubusercontent.com/rohithreddyvennam/AssignmentsFiles/master/sampleSubmission.csv')


Display dataframes

In [0]:
train_df
test_df
submission_df

Drop Extra Columns 

In [0]:
# drop Id axis
train_df_trim = train_df.drop(['Id'], axis = 1)
test_df_trim = test_df.drop(['Id'], axis = 1)

Extract Real and Imaginary

In [0]:
# set precision 
pd.set_option("display.precision", 24)

# changing object type to complex type
df_train_complex = train_df_trim.apply(lambda col: col.apply(lambda val: complex(val.strip('()'))))
train_df_real = df_train_complex.apply(np.real)
train_df_imag = df_train_complex.apply(np.imag)

# changing column names
for ind in range(24):
  train_df_real.rename(index=str, columns={str(ind): (str(ind)+"r")}, inplace=True)
  train_df_imag.rename(index=str, columns={str(ind): (str(ind)+"i")}, inplace=True)

# changing object type to complex type
df_test_complex = test_df_trim.apply(lambda col: col.apply(lambda val: complex(val.strip('()'))))
test_df_real = df_test_complex.apply(np.real)
test_df_imag = df_test_complex.apply(np.imag)

# changing column names
for ind in range(24):
  test_df_real.rename(index=str, columns={str(ind): (str(ind)+"r")}, inplace=True)
  test_df_imag.rename(index=str, columns={str(ind): (str(ind)+"i")}, inplace=True)

# df_train = pd.concat([train_df_real, train_df_imag], axis=1, join='inner')
# df_test = pd.concat([test_df_real, test_df_imag], axis=1, join='inner')
# df1.append(df2)

In [0]:
print (train_df_real)
print (train_df_imag)
print (test_df_real)
print (test_df_imag)

Model

In [0]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [0]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(0, (len(dataset)-look_back),look_back):
    a = dataset[i:(i+look_back), :]
    dataX.append(a)
    dataY.append(dataset[i + look_back, :])
  return np.array(dataX), np.array(dataY)

In [0]:
# Load Data &  train full dataset
dataset_train_real = train_df_real.values
train_real = dataset_train_real.astype('float64')

dataset_test_real = test_df_real.values
test_real = dataset_test_real.astype('float64')

dataset_train_imag = train_df_imag.values
train_imag = dataset_train_imag.astype('float64')

dataset_test_imag = test_df_imag.values
test_imag = dataset_test_imag.astype('float64')

In [140]:
# Display sets
print("train_real :",train_real.shape)
print("test_real :",test_real.shape)
print("train_imag :",train_imag.shape)
print("test_imag :",test_imag.shape)

train_real : (6000, 24)
test_real : (3200, 24)
train_imag : (6000, 24)
test_imag : (3200, 24)


In [0]:
# split into train and test sets
# train_size = int(len(dataset_norm) * 0.75)
# test_size = len(dataset_norm) - train_size
# train, test = dataset_norm[0:train_size,:], dataset_norm[train_size:len(dataset),:]

# no timestamps used to predict 
look_back = 4
samples = train_real.shape[0]
features = train_real.shape[1]

# reshape
trainX_real, trainY_real = create_dataset(train_real, look_back)
testX_real = test_real.reshape(int(test_real.shape[0]/look_back),look_back,test_real.shape[1])
trainX_imag, trainY_imag = create_dataset(train_imag, look_back)
testX_imag = test_imag.reshape(int(test_imag.shape[0]/look_back),look_back,test_imag.shape[1])

In [145]:
print(train_real.shape,trainX_real.shape, trainY_real.shape)
print(test_real.shape,testX_real.shape)
print(train_imag.shape,trainX_imag.shape, trainY_imag.shape)
print(test_imag.shape,testX_imag.shape)
print('samples : ',samples,', features : ',features)

(6000, 24) (1499, 4, 24) (1499, 24)
(3200, 24) (800, 4, 24)
(6000, 24) (1499, 4, 24) (1499, 24)
(3200, 24) (800, 4, 24)
samples :  6000 , features :  24


In [159]:
model_real = Sequential()
model_real.add(LSTM(features, input_shape=(look_back, features)))
model_real.add(Dense(features))
model_real.compile(loss='mean_squared_error', optimizer='adam')
model_real.fit(trainX_real, trainY_real, epochs=100, batch_size=1, verbose=1, shuffle=False)
model_real.summary()

Epoch 1/100
1499/1499 [==============================] - 4s 3ms/step - loss: 0.0070
Epoch 2/100
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0019
Epoch 3/100
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0014
Epoch 4/100
1499/1499 [==============================] - 4s 2ms/step - loss: 0.0011
Epoch 5/100
1499/1499 [==============================] - 3s 2ms/step - loss: 9.7390e-04
Epoch 6/100
1499/1499 [==============================] - 3s 2ms/step - loss: 8.3573e-04
Epoch 7/100
1499/1499 [==============================] - 4s 2ms/step - loss: 7.2373e-04
Epoch 8/100
1499/1499 [==============================] - 3s 2ms/step - loss: 6.2969e-04
Epoch 9/100
1499/1499 [==============================] - 5s 3ms/step - loss: 5.4884e-04
Epoch 10/100
1499/1499 [==============================] - 5s 4ms/step - loss: 4.8093e-04
Epoch 11/100
1499/1499 [==============================] - 5s 3ms/step - loss: 4.2488e-04
Epoch 12/100
1499/1499 [======================

In [158]:
model_imag = Sequential()
model_imag.add(LSTM(features, input_shape=(look_back, features)))
model_imag.add(Dense(features))
model_imag.compile(loss='mean_squared_error', optimizer='adam')
model_imag.fit(trainX_imag, trainY_imag, epochs=100, batch_size=1, verbose=1, shuffle=False)
model_imag.summary()

Epoch 1/100
1499/1499 [==============================] - 4s 3ms/step - loss: 0.0077
Epoch 2/100
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0019
Epoch 3/100
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0014
Epoch 4/100
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0011
Epoch 5/100
1499/1499 [==============================] - 3s 2ms/step - loss: 9.4435e-04
Epoch 6/100
1499/1499 [==============================] - 3s 2ms/step - loss: 8.1702e-04
Epoch 7/100
1499/1499 [==============================] - 3s 2ms/step - loss: 7.1344e-04
Epoch 8/100
1499/1499 [==============================] - 3s 2ms/step - loss: 6.2387e-04
Epoch 9/100
1499/1499 [==============================] - 3s 2ms/step - loss: 5.4606e-04
Epoch 10/100
1499/1499 [==============================] - 3s 2ms/step - loss: 4.7945e-04
Epoch 11/100
1499/1499 [==============================] - 3s 2ms/step - loss: 4.2347e-04
Epoch 12/100
1499/1499 [======================

In [160]:
# make predictions
trainPredict_real = model.predict(trainX_real)
testPredict_real = model.predict(testX_real)

trainPredict_imag = model.predict(trainX_imag)
testPredict_imag = model.predict(testX_imag)


print(trainPredict_real.shape,trainY_real.shape, trainPredict_imag.shape,trainY_imag.shape)
print(testPredict_real.shape,testPredict_imag.shape)

# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
#trainY = scaler.inverse_transform([trainY])
#testPredict = scaler.inverse_transform(testPredict)
#testY = scaler.inverse_transform([testY])


(1499, 24) (1499, 24) (1499, 24) (1499, 24)
(800, 24) (800, 24)


In [161]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY_real[:,:], trainPredict_real[:,:]))
print('Train Score: %.5f RMSE' % (trainScore))
trainScore = math.sqrt(mean_squared_error(trainY_imag[:,:], trainPredict_imag[:,:]))
print('Train Score: %.5f RMSE' % (trainScore))
#testScore = math.sqrt(mean_squared_error(testY[:,:], testPredict[:,:]))
#print('Test Score: %.5f RMSE' % (testScore))

Train Score: 0.01194 RMSE
Train Score: 0.01408 RMSE


In [162]:
print("testPredict_real",testPredict_real)
print("testPredict_imag",testPredict_imag)

testPredict_real [[-0.03001188  0.00297151  0.13133228 ...  0.05652747 -0.05103165
   0.00984133]
 [-0.02329423  0.12372302  0.17992842 ...  0.07066316 -0.06314562
   0.01576314]
 [-0.00420924  0.16702917  0.22249907 ...  0.05312184 -0.06055377
   0.01700628]
 ...
 [ 0.04735734  0.04034699 -0.08762878 ... -0.02657546 -0.00763394
   0.04737229]
 [-0.01995688  0.09008821 -0.15161651 ... -0.01298008 -0.00566526
   0.04479961]
 [-0.06772197  0.19419526 -0.15758178 ...  0.0027257  -0.00361911
   0.03404401]]
testPredict_imag [[ 0.33594584  0.71249026  0.10382327 ...  0.00622371  0.01747202
   0.00238531]
 [ 0.35235542  0.6788337   0.11736022 ...  0.02631615  0.00953578
   0.01517659]
 [ 0.28808174  0.52683026  0.08652071 ...  0.03081469 -0.00209489
   0.02584946]
 ...
 [ 0.02391118  0.09946964  0.17586957 ...  0.01562013 -0.0413981
   0.00142867]
 [ 0.06857813  0.20579813  0.05502083 ...  0.04376975 -0.02850284
  -0.00343068]
 [ 0.08124907  0.27947772 -0.06980911 ...  0.05053839 -0.02063436

In [0]:
np.savetxt('data_real.csv', testPredict_real, delimiter=',')
np.savetxt('data_imag.csv', testPredict_imag, delimiter=',')
